In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

/Users/quansun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X = tf.placeholder(tf.float32,shape=[None,784])

initializer = tf.contrib.layers.xavier_initializer()

D_W1 = tf.Variable(initializer((784,128)))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(initializer((128,1)))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1,D_W2,D_b1,D_b2]

Z = tf.placeholder(tf.float32,shape=[None,100])

G_W1 = tf.Variable(initializer((100,128)))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(initializer((128,784)))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1,G_W2,G_b1,G_b2]

In [3]:
def sample_Z(m,n):
    return np.random.uniform(-1.,1.,size=[m,n])

In [4]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z,G_W1)+G_b1)
    G_log_prob = tf.matmul(G_h1,G_W2)+G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [5]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x,D_W1)+D_b1)
    D_logit = tf.matmul(D_h1,D_W2)+D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob,D_logit

In [6]:
def plot(samples):
    fig = plt.figure(figsize=(4,4))
    gs = gridspec.GridSpec(4,4)
    gs.update(wspace=0.05,hspace=0.05)
    
    for i,sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28,28),cmap='Greys_r')
    return fig

In [8]:
G_sample = generator(Z)

D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real,labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake,labels=tf.zeros_like(D_logit_fake)))

D_loss = D_loss_real + D_loss_fake

G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake,labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss,var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss,var_list=theta_G)

In [11]:
batch_size=128
Z_dim = 100

mnist = input_data.read_data_sets('./mnist',one_hot=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('output/'):
    os.makedirs('output/')
    
i = 0
for iteration in range(20000):
    if iteration % 2000 == 0:
        samples = sess.run(G_sample,feed_dict={Z:sample_Z(16,Z_dim)})
        
        fig = plot(samples)
        plt.savefig('output/{}.jpg'.format(str(i).zfill(3)),bbox_inches='tight')
        i+=1
        plt.close(fig)
        
    X_batch, _ = mnist.train.next_batch(batch_size)
    _,D_loss_current = sess.run([D_solver,D_loss],feed_dict={X:X_batch,Z:sample_Z(batch_size,Z_dim)})
    _,G_loss_current = sess.run([G_solver,G_loss],feed_dict={Z:sample_Z(batch_size,Z_dim)})
    
    if iteration % 2000 == 0:
        print('Iter: {}'.format(iteration))
        print('D_loss: {:.4}'.format(D_loss_current))
        print('G_loss: {:.4}'.format(G_loss_current))

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Iter: 0
D_loss: 1.072
G_loss: 2.652
Iter: 2000
D_loss: 0.03403
G_loss: 6.082
Iter: 4000
D_loss: 0.01975
G_loss: 6.727
Iter: 6000
D_loss: 0.2602
G_loss: 6.076
Iter: 8000
D_loss: 0.3122
G_loss: 4.424
Iter: 10000
D_loss: 0.4966
G_loss: 3.529
Iter: 12000
D_loss: 0.5231
G_loss: 2.986
Iter: 14000
D_loss: 0.5613
G_loss: 2.603
Iter: 16000
D_loss: 0.5687
G_loss: 2.311
Iter: 18000
D_loss: 0.6578
G_loss: 2.589
